<a href="https://colab.research.google.com/github/endalk200/milvus.poc/blob/main/milvus_poc_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Hello world")

Hello world
